This study employs the MIMIC-IV Note dataset, which comprises de-identified clinical data for research purposes. 
The dataset was received via PhysioNet, which is a restricted-access resource.


# Pipeline-1

In [1]:
import modules
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation

# Set the option to display all rows
pd.set_option('display.max_rows', None)

# Set the option to display all columns
pd.set_option('display.max_columns', None)




/Users/saanidhya/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# Run this first

# Extracts Diabetes related clinical notes


#df = pd.read_csv('discharge.csv')
# import re

# # Compile a regex pattern for broader matching
# pattern = re.compile(
#     r'\b(diabetes|diabetic|DM|T1DM|T2DM|type\s+1\s+diabetes|type\s+2\s+diabetes|'
#     r'diabetes\s+mellitus|gestational\s+diabetes|insulin-dependent\s+diabetes|'
#     r'non-insulin-dependent\s+diabetes|adult-onset\s+diabetes|juvenile\s+diabetes)\b',
#     flags=re.IGNORECASE
# )

# # Apply the filter
# df_filtered = df[df['text'].apply(lambda x: bool(pattern.search(x)) if pd.notna(x) else False)]
#df_filtered.to_csv('diabetes.csv')

In [2]:
df_filtered = pd.read_csv('diabetes.csv')

In [3]:
df_filtered.shape

(124315, 9)

In [4]:
df_filtered.head(10)

,Unnamed: 0,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,18,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,\nName: ___ Unit No: ___\n \nAdmi...
1,19,10000980-DS-21,10000980,26913865,DS,21,2189-07-03 00:00:00,2189-07-03 19:50:00,\nName: ___ Unit No: ___\n \nAdmi...
2,20,10000980-DS-22,10000980,24947999,DS,22,2190-11-08 00:00:00,2190-11-09 13:57:00,\nName: ___ Unit No: ___\n \nAdmi...
3,21,10000980-DS-23,10000980,25242409,DS,23,2191-04-11 00:00:00,2191-04-11 17:48:00,\nName: ___ Unit No: ___\n \nAdmi...
4,22,10000980-DS-24,10000980,25911675,DS,24,2191-05-24 00:00:00,2191-05-24 17:29:00,\nName: ___ Unit No: ___\n \nAdmi...
5,23,10000980-DS-25,10000980,29659838,DS,25,2191-07-19 00:00:00,2191-07-22 09:37:00,\nName: ___ Unit No: ___\n \nAdmi...
6,24,10000980-DS-26,10000980,20897796,DS,26,2193-08-17 00:00:00,2193-08-17 16:05:00,\nName: ___ Unit No: ___\n \nAdmi...
7,25,10001176-DS-17,10001176,23334588,DS,17,2186-12-02 00:00:00,2186-12-02 16:38:00,\nName: ___ Unit No: ___...
8,41,10001667-DS-10,10001667,22672901,DS,10,2173-08-24 00:00:00,2173-08-24 15:04:00,\nName: ___ Unit No: ___\n ...
9,44,10001877-DS-19,10001877,25679292,DS,19,2149-05-27 00:00:00,2149-05-27 11:29:00,\nName: ___ ___ No: ___\n \...


In [5]:
# Replace '___' including any surrounding spaces with a single space and then trim excess spaces
df_filtered['text'] = df_filtered['text'].str.replace(r'\s*___\s*', ' ', regex=True).str.strip()

In [6]:
import re
# Function to check for acute or active issues and mark them
def mark_acute_issues(text):
    # Regex pattern to identify sections
    pattern = re.compile(r'(?<=\n)((acute|active)(/acute|/active)?(\s+or\s+(acute|active))?(\s+(issues|problems|conditions)))\s*[:]?([\s\S]*?)(?=\n[A-Z\s]+:|\Z)',re.IGNORECASE | re.MULTILINE)

    # Search for matches
    match = pattern.search(text)
    return 1 if match else 0  # Return 1 if a match is found, else 0


df_filtered['acute_issue_found'] = df_filtered['text'].apply(mark_acute_issues)

In [7]:
df_filtered['acute_issue_found'].value_counts()

#output - acute_issue_found
#0    104006
#1     20309

acute_issue_found
0    104006
1     20309
Name: count, dtype: int64

In [8]:
#remove texts where there's no acute issues or its variations
df_filtered = df_filtered[df_filtered['acute_issue_found'] != 0]

In [ ]:
# In the early stages, when trying to catch all relevant texts concerning acute issues.
# There were certain cases where it would either return too much or too little data
# it was decided that capturing the entire data after acute_issues is found 
# then removing the unwanted headers and text would be more effective
# This was done to preserve relevant information while removing unnecessary content

In [9]:
import pandas as pd
import re

# Define the function to extract text from the mention of acute issues to the end of the document
def extract_acute_issues(text):
    """
    Extracts all text from the mention of acute issues until the end of the document, ignoring any section headers that follow.
 
    """
    # Regex pattern to capture text from "acute issues" or variations until the end of the document
    pattern = re.compile(r'(acute|active)\s+(issues|conditions|problems)(?:[:\s]*)([\s\S]*)', re.IGNORECASE)
    
    # Search for matches
    match = pattern.search(text)
    
    # Return the match if found, otherwise return an empty string
    return match.group(3) if match else ""


df_filtered['acute_issues'] = df_filtered['text'].apply(extract_acute_issues)


In [10]:
# Unwanted sections are now systematically removed 
# Function to clean text and flag
def clean_headers(text):

    # Define the regex pattern inside the function
    exclusion_pattern = re.compile(
        r'(?i)\n('
        r'(INACTIVE|CHRONIC|STABLE|TRANSITIONAL|DISCHARGE|'
        r'MEDICATIONS|FOLLOWUP|RESOLVED)\s*/?\s*'
        r'(ISSUES|PROBLEMS|CONDITIONS|DIAGNOSES|ISSUE|ILLNESSES|ILLNESS)|'
        r'(CHRONIC|STABLE|RESOLVED)\s*/?\s*(CHRONIC|STABLE|RESOLVED)\s*/?\s*(CHRONIC|STABLE|RESOLVED)?\s*/?\s*'
        r'(ISSUES|PROBLEMS|CONDITIONS|DIAGNOSES|ISSUE|ILLNESSES|ILLNESS)|'
        r'CHRONIC\s?&\s?RESOLVED\s*ISSUES|'  
        r'Medications on Admission:|'
        r'DISCHARGE LABS:|'
        r'Past Medical History:|'
        r'PROCEDURE DETAILS:|'
        r'Discharge Medications:|'
        r'CORE ISSUES:|'
        r'CORE\s*MEASURES|'
        r'Treatment Plan|'
        r'Medications for management:'
        r')\s*:?[\s\S]*$', re.DOTALL)
    
    # Search for the pattern in the text
    match = exclusion_pattern.search(text)
    if match:
        return text[:match.start()], 1
    # Return the original text and flag 0 if no pattern is found
    return text, 0


df_filtered[['cleaned_text', 'flag']] = df_filtered['acute_issues'].apply(clean_headers).apply(pd.Series)

In [11]:
# Still some unwanted headers had to be removed
import re

def remove_remaining_headers(text):
    # Lowercase the text 
    text = text.lower()
    
    pattern = re.compile(r'^#(transitional issues|code status|contact|hcp/contact|code)', flags=re.MULTILINE)

    # Search for the pattern
    match = pattern.search(text)
    if match:
        # Find the index where the matched line starts
        cut_off_index = match.start()
        # Return the string from the start of the text to just before the matched line
        return text[:cut_off_index]
    return text

df_filtered['cleaned_text'] = df_filtered['cleaned_text'].apply(remove_remaining_headers)

# preprocessing 

In [12]:
import re

def remove_leading_period(text):
    # Regex pattern to match a newline followed by any amount of whitespace and a period
    # The pattern also includes any trailing whitespace after the period
    pattern = re.compile(r'\n\s*\.\s*')
    
    # Replace the found pattern with just a newline 
    # This effectively removes the period and maintains the whitespace formatting
    cleaned_text = re.sub(pattern, '\n', text)
    
    return cleaned_text

df_filtered['cleaned_text'] = df_filtered['cleaned_text'].apply(remove_leading_period)

In [13]:
#cleaning lines with only special characters
#lines such as ######## 
import re

def remove_special_characters(text):
    lines = text.split('\n')
    # Identify indices where lines consist solely of special characters
    special_char_lines = [i for i, line in enumerate(lines) if re.match(r'^[\W_]+$', line.strip())]

    if len(special_char_lines) > 1:
        # Get the index of the second occurrence of a special character line
        second_occurrence = special_char_lines[1]
        # Return everything up to but not including the line above the second occurrence
        return '\n'.join(lines[:max(0, second_occurrence - 1)])
    elif len(special_char_lines) == 1:
        # If there's only one occurrence, remove it
        return '\n'.join(line for i, line in enumerate(lines) if i != special_char_lines[0])
    else:
        # If there are no special character lines, return the original text
        return text

def preprocess_text(text):
    
    final_text = remove_special_characters(text)
    # Second pass: remove any remaining lines with only special characters if necessary
    final_text = '\n'.join(line for line in final_text.split('\n') if not re.match(r'^[\W_]+$', line.strip()))
    return final_text

df_filtered['transcript'] = df_filtered['cleaned_text'].apply(preprocess_text)

In [14]:
# remove numerical index 
def remove_numerical_indices(text):
    # Regex to match numbers at the start of a line followed by '.' or ')', ensuring it matches whole indices
    # ^\d+[\.\)] ensures matching numbers that are indices (like "1." or "2)")
    # Added \s+ to ensure that it's followed by a space (which helps confirm it's an index)
    text = re.sub(r'^\d+[\.\)]\s+', '', text, flags=re.MULTILINE)
    return text
df_filtered['transcript'] = df_filtered['transcript'].apply(remove_numerical_indices)

In [15]:
def remove_hyphens(text):
    # Regex pattern to match a hyphen at the start of any line, preceded by whitespace
    pattern = re.compile(r'(?m)^\s*-')
    
   #remove the pattern
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
df_filtered['transcript'] = df_filtered['transcript'].apply(remove_hyphens)

In [16]:
# Function to extract diagnosis based on the pattern
# Usally most of the diagnosis are followed by clincal sentences
# In this study, as the focus is only on clinical sentences; diagnosis aren't needed
def extract_diagnosis(text):
    pattern = re.compile(r'(^#.*?:|^#.*$|^[^#\n]+?:)', re.MULTILINE)
    matches = pattern.findall(text)
    headings = []
    seen = set()
    for match in matches:
        heading = match.strip()
        if heading and heading not in seen:
            seen.add(heading)
            headings.append(heading)
    return '\n'.join(headings), pattern.sub('', text).strip()


df_filtered[['diagnosis', 'transcript']] = df_filtered['transcript'].apply(lambda x: pd.Series(extract_diagnosis(x)))

In [17]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation



# Function to preprocess a single sentence
def preprocess_sentence(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in punctuation and token not in stop_words]

    # Join tokens back into a single string and return
    return ' '.join(tokens)

# Function to split the text into sentences
def preprocess_text(text):
    sentences = sent_tokenize(text)  # Split into sentences
    return sentences

# Apply preprocessing to the transcript column to split into sentences and remove stopwords
df_filtered['transcript'] = df_filtered['transcript'].apply(
    lambda text: [preprocess_sentence(sentence) for sentence in preprocess_text(text)]
)


In [18]:
#remove unnecessary columns
# List of columns to drop
columns_to_drop = ['acute_issue_found', 'acute_issues', 'cleaned_text', 'flag']

# Drop the specified columns
df_filtered.drop(columns=columns_to_drop, axis=1, inplace=True)

# Display the updated DataFrame
print(df_filtered.info())

<class 'pandas.core.frame.DataFrame'>
Index: 20309 entries, 12 to 124313
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20309 non-null  int64 
 1   note_id     20309 non-null  object
 2   subject_id  20309 non-null  int64 
 3   hadm_id     20309 non-null  int64 
 4   note_type   20309 non-null  object
 5   note_seq    20309 non-null  int64 
 6   charttime   20309 non-null  object
 7   storetime   20309 non-null  object
 8   text        20309 non-null  object
 9   transcript  20309 non-null  object
 10  diagnosis   20309 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.9+ MB
None


In [19]:
# Validating the results before further analysis
print(df_filtered['transcript'].head())  # Inspect the first few entries
print(df_filtered['transcript'].dtype) 

12    [patient presented without history urinary sys...
20    [patient high coronary risk however recent cat...
25    [pt endorsed 4mths pain rle became acutely wor...
26    [patient presents severe right lower back pain...
29    [placement presented inflammatory pericarditis...
Name: transcript, dtype: object
object


In [21]:
# Verify the type and content of the 'transcript' column
#  each entry in 'transcript' should be a list
print("Data type of 'transcript' column:", type(df_filtered['transcript'].iloc[0]))
print("Content of the first entry in 'transcript' column:", df_filtered['transcript'].iloc[0])




Data type of 'transcript' column: <class 'list'>
Content of the first entry in 'transcript' column: ['patient presented without history urinary systemic symptoms started ceftriaxone ed u/a positive 19 wbcs', 'antibiotics taken prior drawing urinary blood cultures yield', 'patient switched ciprofloxacin received three day total antibiotic course', 'ct scan performed evidence pyelonephritis', 'antibiotics discontinued time discharge', 'patient reported 3 weeks back/flank pain constant achy nature worsened movement', 'treated anti-inflammatories minimal effect', 'ct scan demonstrated nephrolithiasis', 'cxr showed bony abnormality could totally exclude multiple rib fractures', "patient 's pain well controlled tolerating po medications discharged pcp following workup", 'patient persistently last a1c 8.0. serum glucose initially 400s chem-7 gap however likely lactate unlikely dka given normal ph abg', 'glucose 218. continue home dose lantus 90 units qpm per records aggressive iss decrease up

In [22]:
# Create an empty list to hold individual sentences
all_sentences = []

# Iterate over each row in the DataFrame
for index, row in df_filtered.iterrows():
    # Iterate over each sentence in 'transcript' list
    for sentence in row['transcript']:
        # Append each sentence to the all_sentences list along with an index for reference
        all_sentences.append({'original_index': index, 'sentence': sentence})

# Create a new DataFrame from the list of sentences
df_sentences = pd.DataFrame(all_sentences)

In [23]:
# returns 441,492 rows of clinical sentences
df_sentences.shape

(441492, 2)

In [25]:
# Ensuring the sentence column is of type string
df_sentences['sentence'] = df_sentences['sentence'].astype(str)

In [26]:
# Numbers could've influenced the semantic meaning of the sentences
# In this case, number with compund units are removed so that measurements dont distort the results
import re

def preprocess_numbers(text):
    # Pattern to remove numbers with units and compound units
    pattern = r"\b\d+\.?\d*\s*(mg/kg|mg/dl|mg/L|ml/kg|mg|kg|ml|g|l|units|cm|mm|g/l|gnr|gpc)\b|\b\d+\.\d+\b|\b\d+\b"
    
    # Remove matching patterns
    text = re.sub(pattern, "", text)
    
    # Remove special characters
    text = re.sub(r"[‘~\\\+/\.-]", "", text)
    
    # Clean up extra spaces left after removals
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

df_sentences['sentence'] = df_sentences['sentence'].apply(preprocess_numbers)


In [27]:
def cleaning_possesives(text):
    # Remove instances of "'s" without relying on word boundaries
    text = re.sub(r"'s\b", "", text)  # Matches 's at the end of a word
    
    # Remove instances of "`" followed by "'" around words
    text = re.sub(r"`\s*'\s*", "", text)
    
    # Remove instances of "''" around any word
    text = re.sub(r"\s*''\s*", " ", text)
    
    # Clean up any extra spaces left after removals
    text = re.sub(r"\s+", " ", text).strip()
    
    return text
df_sentences['sentence'] = df_sentences['sentence'].apply(cleaning_possesives)

In [28]:
###  removing short sentences

#short sentences usually lack context or semantic meaning

# Count the number of sentences with fewer than 3 words

# These may negatively impact the clustering

short_sentences_count = df_sentences['sentence'].apply(lambda x: len(x.split()) <= 3).sum()

print(f"Number of sentences with fewer than 3 words: {short_sentences_count}")

# output: Number of sentences with fewer than 3 words: 56120

Number of sentences with fewer than 3 words: 56120


In [29]:
# Filter out sentences with 3 or fewer words
df_sentences_filtered = df_sentences[df_sentences['sentence'].apply(lambda x: len(x.split()) > 3)]

# Check the number of rows after filtering
print(f"Number of sentences after filtering: {len(df_sentences_filtered)}")

Number of sentences after filtering: 385372


In [30]:
# duplicate sentences are removed as they can skew the results later\

# Drop duplicate sentences
df_sentences_filtered = df_sentences_filtered.drop_duplicates(subset='sentence', keep='first')

# Reset the index after dropping duplicates 
df_sentences_filtered = df_sentences_filtered.reset_index(drop=True)

# Display the number of remaining sentences
print(f"Number of unique sentences: {len(df_sentences_filtered)}")

#Number of unique sentences: 380292

Number of unique sentences: 380292


In [ ]:
#Save the file for embedding generation

df_sentences_filtered.to_csv('sentences_vf.csv')